<a href="https://colab.research.google.com/github/jojosam90/imagesegmentation/blob/main/maskrcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow as tf
tf.__version__

'1.13.1'

In [6]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

Using TensorFlow backend.


In [36]:
ROOT_DIR = "/content/Mask_RCNN"
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
WEIGHTS_PATH = "/content/drive/MyDrive/dataset/mask_rcnn_coco.h5"  

In [37]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "MaskRCNN_config"

    IMAGES_PER_GPU = 1

    NUM_CLASSES = 1 + 2  # Background + phone,laptop and mobile

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    LEARNING_RATE=0.006

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.4

    # Max ground truth instances
    MAX_GT_INSTANCES=10

config = CustomConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.4
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.006
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [38]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Horse-Man dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """

        self.add_class("object", 1, "chip")
        self.add_class("object", 2, "crack")

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        annotations1 = json.load(open(os.path.join(dataset_dir, "via_project.json")))
        annotations = list(annotations1.values())

        annotations = [a for a in annotations if a['regions']]
        
        for a in annotations:
            polygons = [a['regions']['0']['shape_attributes']] 
            objects = [a['regions']['0']['region_attributes']['label']]
            print("objects:", objects)
            name_dict = {"chip": 1, "crack": 2} #,"xyz": 3}
            num_ids = [name_dict[a] for a in objects]
     
            print("numids", num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object", 
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """

        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):

        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [39]:
# Inspect the model in training or inference modes values: 'inference' or 'training'
TEST_MODE = "inference"
ROOT_DIR = "/content/Mask_RCNN"

def get_ax(rows=1, cols=1, size=16):
  """Return a Matplotlib Axes array to be used in all visualizations in the notebook.  Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
  return ax

# Load validation dataset
# Must call before using the dataset
CUSTOM_DIR = "/content/drive/MyDrive/dataset"
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR, "train")
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['crack']
numids [2]
objects: ['crack']
numids [2]
objects: ['crack']
numids [2]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['crack']
numids [2]
objects: ['crack']
numids [2]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
Images: 25
Classes: ['BG', 'chip', 'crack']


In [40]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')

In [41]:
model.load_weights('/content/drive/MyDrive/dataset/mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [42]:
model.keras_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_image (InputLayer)         (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D) (None, None, None, 3) 0           input_image[0][0]                
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        zero_padding2d_3[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNorm)             (None, None, None, 64 256         conv1[0][0]                      
___________________________________________________________________________________________

In [43]:
IMG_HEIGHT = 1024
IMG_WIDTH = 1024

In [45]:
train_set = CustomDataset()
train_set.load_custom('/content/drive/MyDrive/dataset', 'train')
train_set.prepare()

objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['crack']
numids [2]
objects: ['crack']
numids [2]
objects: ['crack']
numids [2]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['crack']
numids [2]
objects: ['crack']
numids [2]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]


In [53]:
test_set = CustomDataset()
test_set.load_custom('/content/drive/MyDrive/dataset', 'val')
test_set.prepare()

objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]
objects: ['chip']
numids [1]


In [54]:
EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

In [56]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8,
                              patience=10, min_lr=0.0001)

In [ ]:
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=1000, layers='heads', custom_callbacks=[es, reduce_lr])
history = model.keras_model.history.history

In [ ]:
import time
model_path = '/content/drive/MyDrive/diode/mask_rcnn_'  + '.' + str(time.time()) + '.h5'
model.keras_model.save_weights(model_path)

In [ ]:
model = modellib.MaskRCNN(mode="inference", config=config, model_dir='./')

In [ ]:
model.load_weights('/content/Mask_RCNN/maskrcnn_config20220517T0632/mask_rcnn_maskrcnn_config_0127.h5', by_name=True)

In [ ]:
model.load_weights(model_path, by_name=True)